In [1]:
import socket
import pickle
import struct
import copy
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

In [3]:
device = "cpu"

torch.manual_seed(777)
if device =="cuda:0":
    torch.cuda.manual_seed_all(777)

In [4]:
with (open("recsys_data/data.p", "rb")) as openfile:
    train_data = pickle.load(openfile)
with (open("recsys_data/lab_ratings.p", "rb")) as openfile:
    lab_ratings = pickle.load(openfile)
with (open("recsys_data/fb_data_train.p", "rb")) as openfile:
    fb_data_train = pickle.load(openfile)
with (open("recsys_data/fb_labels_train.p", "rb")) as openfile:
    fb_labels_train = pickle.load(openfile)
with (open("recsys_data/fb_data_test.p", 'rb')) as openfile:
    fb_data_test = pickle.load(openfile)
with (open("recsys_data/fb_labels_test.p", "rb")) as openfile:
    fb_labels_test = pickle.load(openfile)

In [5]:
EMBED_FILE = 'recsys_data/processed_full.embed'

In [6]:
def load_embeddings(embed_file):
    #also normalizes the embeddings
    W = []
    with open(embed_file) as ef:
        for line in ef:
            line = line.rstrip().split()
            vec = np.array(line[1:]).astype(np.float)
            vec = vec / float(np.linalg.norm(vec) + 1e-6)
            W.append(vec)
        #UNK embedding, gaussian randomly initialized 
        print("adding unk embedding")
        vec = np.random.randn(len(W[-1]))
        vec = vec / float(np.linalg.norm(vec) + 1e-6)
        W.append(vec)
    W = np.array(W)
    return W

In [7]:
embedding_matrix = load_embeddings(EMBED_FILE)
# embedding_matrix.shape()

adding unk embedding


In [8]:
import pandas as pd
vocab = pd.read_csv('recsys_data/vocab.csv', header=None)
len(set(sorted(vocab[0].tolist())))

51917

In [9]:
def load_vocab_dict(vocab_file):
    vocab_df = pd.read_csv(vocab_file, header=None)
    vocab = sorted(set(vocab_df[0].tolist()))
    ind2w = {i+1:w for i,w in enumerate(vocab)}
    w2ind = {w:i for i,w in ind2w.items()}
    return ind2w, w2ind

In [10]:
idx2w, w2idx = load_vocab_dict('recsys_data/vocab.csv')

In [11]:
def clean_text(text):
    s = text.replace('[', "")
    s = s.replace(']', "")
    s = s.replace("'", "")
    s = s.replace(",", "")
    s = s.split()
    return s

def encoding_disease(data, w2idx):
    idx_total = []
    for i in range(len(data['age'])):
        text = data['disease'][i]
        cleaned_text = clean_text(text)
        idx = []
        for st in cleaned_text:
            if st not in w2idx:
                idx.append(len(w2idx)+1)
            else:
                idx.append(w2idx[st])
        idx_total.append(idx)
    return np.array(idx_total)

In [12]:
def pad_sequences(sequences, max_seq_len: int = 0):
    max_seq_len = max(max_seq_len, max(len(sequence) for sequence in sequences))
    # Pad
    padded_sequences = np.zeros((len(sequences), max_seq_len))
    for i, sequence in enumerate(sequences):
        padded_sequences[i][: len(sequence)] = sequence
    return padded_sequences


In [13]:
train_encoded_disease = encoding_disease(train_data, w2idx)
train_padded_encoded_disease = pad_sequences(train_encoded_disease)
fb_train_encoded_disease = encoding_disease(fb_data_train, w2idx)
fb_train_padded_encoded_disease = pad_sequences(fb_train_encoded_disease)
fb_test_encoded_disease = encoding_disease(fb_data_test, w2idx)
fb_test_padded_encoded_disease = pad_sequences(fb_test_encoded_disease)

train_data['encoded_disease'] = train_padded_encoded_disease
fb_data_train['encoded_disease'] = fb_train_padded_encoded_disease
fb_data_test['encoded_disease'] = fb_test_padded_encoded_disease

<ipython-input-11-1e1034462875>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(idx_total)


In [14]:
EPOCHS = 50
users = 2

In [15]:
train_data['age'] = train_data['age'].reshape((-1, 1))
train_data['weight'] = train_data['weight'].reshape((-1, 1))

In [16]:
scaler_age = MinMaxScaler()
scaler_weight = MinMaxScaler()
train_data['age'] = scaler_age.fit_transform(train_data['age'])
train_data['weight'] = scaler_weight.fit_transform(train_data['weight'])

In [17]:
class InitialDataset(Dataset):
    def __init__(self):
        self.data = train_data
        self.ratings = lab_ratings
    def __getitem__(self, index):
        age = self.data['age'][index]
        weight = self.data['weight'][index]
        icd_codes = self.data['codes'][index]
        disease = self.data['encoded_disease'][index]
        target = self.ratings[index]
        return {
            'age': torch.tensor(age, dtype=float),
            'weight': torch.tensor(weight, dtype=float),
            'disease': torch.tensor(disease, dtype=torch.long),
            'icd_codes': torch.tensor(icd_codes, dtype=float),
            'target': torch.tensor(target, dtype=float)
        }
    def __len__(self):
        return len(self.data['hadm_id'])

In [18]:
fb_data_train['age'] = fb_data_train['age'].reshape((-1, 1))
fb_data_train['weight'] = fb_data_train['weight'].reshape((-1, 1))

scaler_age = MinMaxScaler()
scaler_weight = MinMaxScaler()
fb_data_train['age'] = scaler_age.fit_transform(fb_data_train['age'])
fb_data_train['weight'] = scaler_weight.fit_transform(fb_data_train['weight'])

In [19]:
fb_data_test['age'] = fb_data_test['age'].reshape((-1, 1))
fb_data_test['weight'] = fb_data_test['weight'].reshape((-1, 1))

scaler_age = MinMaxScaler()
scaler_weight = MinMaxScaler()
fb_data_test['age'] = scaler_age.fit_transform(fb_data_test['age'])
fb_data_test['weight'] = scaler_weight.fit_transform(fb_data_test['weight'])

In [20]:
class FeedbackDataset(Dataset):
    def __init__(self, train=False):
        if train:
            self.data = fb_data_train
            self.ratings = fb_labels_train
        else:
            self.data = fb_data_test
            self.ratings = fb_labels_test
            
    def __getitem__(self, index):
        age = self.data['age'][index]
        weight = self.data['weight'][index]
        icd_codes = self.data['codes'][index]
        disease = self.data['encoded_disease'][index]
        target = self.ratings[index]
        return {
            'age': torch.tensor(age, dtype=float),
            'weight': torch.tensor(weight, dtype=float),
            'disease': torch.tensor(disease, dtype=torch.long),
            'icd_codes': torch.tensor(icd_codes, dtype=float),
            'target': torch.tensor(target, dtype=float)
        }
    def __len__(self):
        return len(self.data['age'])

In [21]:
initial_dataset = InitialDataset()
train_loader = DataLoader(initial_dataset, batch_size=64, shuffle=True)

fb_train_dataset = FeedbackDataset(train=True)
fb_train_loader = DataLoader(fb_train_dataset, batch_size=64, shuffle=True)

fb_test_dataset = FeedbackDataset(train=False)
fb_test_loader = DataLoader(fb_test_dataset, batch_size=64)

In [22]:
class RecSysServer(nn.Module):
    def __init__(self):
        super().__init__()
#             self.fc1 = nn.Linear(52, 1024)
#             self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 25)
    def forward(self, x):
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x
            

In [23]:
server = RecSysServer()
print(server)

RecSysServer(
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=25, bias=True)
)


In [24]:
class RecSysClient(nn.Module):
    def __init__(self):
        super().__init__()
#         W = torch.Tensor(load_embeddings(EMBED_FILE))
#         self.embed = nn.Embedding(W.size()[0], W.size()[1], padding_idx=0)
#         self.embed.weight.data = W.clone()
        W = torch.Tensor(embedding_matrix)
        self.embed = nn.Embedding(W.size()[0], W.size()[1], padding_idx=0)
        self.embed.weight.data = W.clone()
        
        self.fc1 = nn.Linear(152, 1024)
        self.fc2 = nn.Linear(1024, 512)
#         self.fc3 = nn.Linear(512, 256)
#         self.fc4 = nn.Linear(256, 25)
    def forward(self, age, weight, icd_codes, disease):
#         age = x['age']
#         weight = x['weight']
#         icd_codes = x['icd_codes']
        
        embedded = self.embed(disease)
        embedded = torch.mean(embedded, 1)
        
        x = torch.cat((embedded, icd_codes, age, weight), 1).float()
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        return x

In [25]:
client = RecSysClient()
print(client)

RecSysClient(
  (embed): Embedding(51919, 100, padding_idx=0)
  (fc1): Linear(in_features=152, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
)


In [26]:
criterion = nn.MSELoss()
lr = 0.001
optimizer_server = Adam(server.parameters(), lr=lr)

clientsoclist = []
train_total_batch = []
client_weights = copy.deepcopy(client.state_dict())

In [27]:
total_sendsize_list = []
total_receivesize_list = []

client_sendsize_list = [[] for i in range(users)]
client_receivesize_list = [[] for i in range(users)]

train_sendsize_list = [] 
train_receivesize_list = []

In [28]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    l_send = len(msg)
    msg = struct.pack('>I', l_send) + msg
    sock.sendall(msg)
    return l_send

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg, msglen

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

In [29]:
host = socket.gethostbyname(socket.gethostname())
port = 10080
print(host)

10.10.7.64


In [30]:
s = socket.socket()
s.bind((host, port))
s.listen(5)

In [31]:
for i in range(users):
    conn, addr = s.accept()
    print('Conntected with', addr)
    clientsoclist.append(conn)    # append client socket on list

    datasize = send_msg(conn, EPOCHS)    #send epoch
    total_sendsize_list.append(datasize)
    client_sendsize_list[i].append(datasize)

    total_batch, datasize = recv_msg(conn)    # get total_batch of train dataset
    total_receivesize_list.append(datasize)
    client_receivesize_list[i].append(datasize)

    train_total_batch.append(total_batch)    # append on list

Conntected with ('10.10.7.64', 62701)
Conntected with ('10.10.7.64', 62702)


In [32]:
for e in range(EPOCHS):

    # train client 0

    for user in range(users):

        datasize = send_msg(clientsoclist[user], client_weights)
        total_sendsize_list.append(datasize)
        client_sendsize_list[user].append(datasize)
        train_sendsize_list.append(datasize)

        for i in tqdm(range(train_total_batch[user]), ncols=100, desc='Epoch {} Client{} '.format(e+1, user)):
            optimizer_server.zero_grad()  # initialize all gradients to zero

            msg, datasize = recv_msg(clientsoclist[user])  # receive client message from socket
            total_receivesize_list.append(datasize)
            client_receivesize_list[user].append(datasize)
            train_receivesize_list.append(datasize)

            client_output_cpu = msg['client_output']  # client output tensor
            label = msg['label']  # label

            client_output = client_output_cpu.to(device)
            label = label.clone().detach().float().to(device)

            output = server(client_output)  # forward propagation
            loss = criterion(output, label)  # calculates cross-entropy loss
            loss.backward()  # backward propagation
            msg = client_output_cpu.grad.clone().detach()

            datasize = send_msg(clientsoclist[user], msg)
            total_sendsize_list.append(datasize)
            client_sendsize_list[user].append(datasize)
            train_sendsize_list.append(datasize)
            
            optimizer_server.step()
            
        client_weights, datasize = recv_msg(clientsoclist[user])
        total_receivesize_list.append(datasize)
        client_receivesize_list[user].append(datasize)
        train_receivesize_list.append(datasize)
        
        

    client.load_state_dict(client_weights)
    client.to(device)
    client.eval()


    # train loss
    with torch.no_grad():
        train_loss = 0.0
        for j, trn in enumerate(train_loader):
            trn_age, trn_weight, trn_icd_codes, trn_disease= trn['age'], trn['weight'], trn['icd_codes'], trn['disease']
            trn_target = trn['target']
            
            trn_age = trn_age.to(device)
            trn_weight = trn_weight.to(device)
            trn_icd_codes = trn_icd_codes.to(device)
            trn_disease = trn_disease.to(device)
            trn_target = trn_target.to(device)

            trn_output = client(trn_age, trn_weight, trn_icd_codes, trn_disease)
            trn_output = server(trn_output)
            trn_target = trn_target.float()
            loss = criterion(trn_output, trn_target)
            train_loss += loss.item()

        r_train_loss = train_loss / len(train_loader)
        print("train_loss: {:.4f}".format(r_train_loss))

Epoch 2 Client0 :   0%|                                                     | 0/181 [00:00<?, ?it/s]

train_loss: 5.7572


Epoch 3 Client0 :   0%|                                                     | 0/181 [00:00<?, ?it/s]

train_loss: 5.6990


Epoch 4 Client0 :   0%|                                                     | 0/181 [00:00<?, ?it/s]

...

train_loss: 0.7425


Epoch 50 Client0 :   0%|                                                    | 0/181 [00:00<?, ?it/s]

train_loss: 0.7332


Epoch 50 Client1 : 100%|██████████████████████████████████████████| 181/181 [00:06<00:00, 26.02it/s]


train_loss: 0.6945


In [33]:
# host = socket.gethostbyname(socket.gethostname())
# port = 10080
# print(host)

## Train Feedback dataset

In [34]:
train_total_batch = []

total_sendsize_list = []
total_receivesize_list = []

client_sendsize_list = [[] for i in range(users)]
client_receivesize_list = [[] for i in range(users)]

train_sendsize_list = [] 
train_receivesize_list = []

In [35]:
for i in range(users):

    datasize = send_msg(clientsoclist[i], EPOCHS)    #send epoch
    total_sendsize_list.append(datasize)
    client_sendsize_list[i].append(datasize)

    total_batch, datasize = recv_msg(clientsoclist[i])    # get total_batch of train dataset
    total_receivesize_list.append(datasize)
    client_receivesize_list[i].append(datasize)

    train_total_batch.append(total_batch)    # append on list

In [36]:
for e in range(EPOCHS):

    # train client 0

    for user in range(users):

        datasize = send_msg(clientsoclist[user], client_weights)
        total_sendsize_list.append(datasize)
        client_sendsize_list[user].append(datasize)
        train_sendsize_list.append(datasize)

        for i in tqdm(range(train_total_batch[user]), ncols=100, desc='Epoch {} Client{} '.format(e+1, user)):
            optimizer_server.zero_grad()  # initialize all gradients to zero

            msg, datasize = recv_msg(clientsoclist[user])  # receive client message from socket
            total_receivesize_list.append(datasize)
            client_receivesize_list[user].append(datasize)
            train_receivesize_list.append(datasize)

            client_output_cpu = msg['client_output']  # client output tensor
            label = msg['label']  # label

            client_output = client_output_cpu.to(device)
            label = label.clone().detach().float().to(device)

            output = server(client_output)  # forward propagation
            loss = criterion(output, label)  # calculates cross-entropy loss
            loss.backward()  # backward propagation
            msg = client_output_cpu.grad.clone().detach()

            datasize = send_msg(clientsoclist[user], msg)
            total_sendsize_list.append(datasize)
            client_sendsize_list[user].append(datasize)
            train_sendsize_list.append(datasize)
            
            optimizer_server.step()
            
        client_weights, datasize = recv_msg(clientsoclist[user])
        total_receivesize_list.append(datasize)
        client_receivesize_list[user].append(datasize)
        train_receivesize_list.append(datasize)
        
        

    client.load_state_dict(client_weights)
    client.to(device)
    client.eval()


    # train acc
    with torch.no_grad():
        train_loss = 0.0
        for j, trn in enumerate(fb_train_loader):
            trn_age, trn_weight, trn_icd_codes, trn_disease = trn['age'], trn['weight'], trn['icd_codes'], trn['disease']
            trn_target = trn['target']
            
            trn_age = trn_age.to(device)
            trn_weight = trn_weight.to(device)
            trn_icd_codes = trn_icd_codes.to(device)
            trn_disease = trn_disease.to(device)
            
            trn_target = trn_target.to(device)

            trn_output = client(trn_age, trn_weight, trn_icd_codes, trn_disease)
            trn_output = server(trn_output)
            trn_target = trn_target.float()
            loss = criterion(trn_output, trn_target)
            train_loss += loss.item()

        r_train_loss = train_loss / len(train_loader)
        print("train_loss: {:.4f}".format(r_train_loss))
    
    # test acc
    with torch.no_grad():
        val_loss = 0.0
        for j, val in enumerate(fb_test_loader):
            val_age, val_weight, val_icd_codes, val_disease = val['age'], val['weight'], val['icd_codes'], val['disease']
            val_target = val['target']
            
            val_age = val_age.to(device)
            val_weight = val_weight.to(device)
            val_icd_codes = val_icd_codes.to(device)
            val_disease = val_disease.to(device)
            
            val_target = val_target.to(device)

            val_output = client(val_age, val_weight, val_icd_codes, val_disease)
            val_output = server(val_output)
        
            val_label = val_target.float()
            loss = criterion(val_output, val_label)
            val_loss += loss.item()
            
        test_loss = val_loss / len(fb_test_loader)
        print("test_loss: {:.4f}".format(test_loss))

Epoch 1 Client1 : 100%|█████████████████████████████████████████████| 73/73 [00:02<00:00, 25.92it/s]


train_loss: 3.7204


Epoch 2 Client0 :   0%|                                                      | 0/73 [00:00<?, ?it/s]

test_loss: 4.7099


Epoch 2 Client1 : 100%|█████████████████████████████████████████████| 73/73 [00:02<00:00, 25.24it/s]


...

train_loss: 0.1254


Epoch 50 Client0 :   3%|█▏                                           | 2/73 [00:00<00:04, 17.71it/s]

test_loss: 0.6366


Epoch 50 Client1 : 100%|████████████████████████████████████████████| 73/73 [00:02<00:00, 25.93it/s]


train_loss: 0.1274
test_loss: 0.6309


In [37]:
def precision_at_k(y_true, y_pred, k):
    topk_true = np.flip(np.argsort(y_true), 1)[:, :k]
    topk_pred = np.flip(np.argsort(y_pred), 1)[:, :k]

    n_relevant = 0
    n_recommend = 0

    for t, p in zip(topk_true, topk_pred):
        # print(f"t:{t}")
        # print(f"p:{p}")
        n_relevant += len(np.intersect1d(t, p))
        # print(f"rev:{n_relevant}")
        n_recommend += len(p)
        # print(n_recommend)

    return float(n_relevant) / n_recommend

In [38]:
with torch.no_grad():
        val_loss = 0.0
        pred = torch.empty((0, 25))
        for j, val in enumerate(fb_test_loader):
            val_age, val_weight, val_icd_codes, val_disease = val['age'], val['weight'], val['icd_codes'], val['disease']
            val_target = val['target']
            
            val_age = val_age.to(device)
            val_weight = val_weight.to(device)
            val_icd_codes = val_icd_codes.to(device)
            val_disease = val_disease.to(device)
            
            val_target = val_target.to(device)

            val_output = client(val_age, val_weight, val_icd_codes, val_disease)
            val_output = server(val_output)
        
            val_label = val_target.float()
            loss = criterion(val_output, val_label)
            val_loss += loss.item()
            pred = torch.cat((pred, val_output))
            
        test_loss = val_loss / len(fb_test_loader)
        print("test_loss: {:.4f}".format(test_loss))

test_loss: 0.6309


In [39]:
pred = pred.numpy()
print(f"After training feedback dataset Precision@5: {precision_at_k(fb_labels_test, pred, 5)}")

After training feedback dataset Precision@5: 0.8190189328743546
